In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For examasdfple, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/notebook294d5a41bc-f62f18-gpt/__results__.html
/kaggle/input/notebook294d5a41bc-f62f18-gpt/__notebook__.ipynb
/kaggle/input/notebook294d5a41bc-f62f18-gpt/__output__.json
/kaggle/input/notebook294d5a41bc-f62f18-gpt/custom.css
/kaggle/input/notebook294d5a41bc-f62f18-gpt/wandb/run-20201126_072037-2nr90d5p/run-2nr90d5p.wandb
/kaggle/input/notebook294d5a41bc-f62f18-gpt/wandb/run-20201126_072037-2nr90d5p/logs/debug.log
/kaggle/input/notebook294d5a41bc-f62f18-gpt/wandb/run-20201126_072037-2nr90d5p/logs/debug-internal.log
/kaggle/input/notebook294d5a41bc-f62f18-gpt/wandb/run-20201126_072037-2nr90d5p/files/wandb-summary.json
/kaggle/input/notebook294d5a41bc-f62f18-gpt/wandb/run-20201126_072037-2nr90d5p/files/config.yaml
/kaggle/input/notebook294d5a41bc-f62f18-gpt/wandb/run-20201126_072037-2nr90d5p/files/output.log
/kaggle/input/notebook294d5a41bc-f62f18-gpt/wandb/run-20201126_072037-2nr90d5p/files/requirements.txt
/kaggle/input/notebook294d5a41bc-f62f18-gpt/wandb/run-20201126_0720

In [2]:
!unzip /kaggle/input/jigsaw-toxic-comment-classification-challenge/*

Archive:  /kaggle/input/jigsaw-toxic-comment-classification-challenge/sample_submission.csv.zip
caution: filename not matched:  /kaggle/input/jigsaw-toxic-comment-classification-challenge/test.csv.zip
caution: filename not matched:  /kaggle/input/jigsaw-toxic-comment-classification-challenge/test_labels.csv.zip
caution: filename not matched:  /kaggle/input/jigsaw-toxic-comment-classification-challenge/train.csv.zip


In [3]:
# !pip install transformers > /dev/null

In [4]:
import pandas as pd, numpy as np
# from sklearn.linear_model import LogisticRegression
# from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import torch
import torch.nn as nn
# import tensorflow as tf
# tf.summary.FileWriter('log_dir') 
from transformers import AutoTokenizer, AutoModel, AdamW, get_linear_schedule_with_warmup
# import wandb
# os.environ["WANDB_API_KEY"] = '3ace0586afdcb3d308f9316e385f913a9463bdaa'

wandb: WARNING W&B installed but not logged in.  Run `wandb login` or set the WANDB_API_KEY env variable.


In [5]:
train = pd.read_csv('/kaggle/input/jigsaw-toxic-comment-classification-challenge/train.csv.zip')
test = pd.read_csv('/kaggle/input/jigsaw-toxic-comment-classification-challenge/test.csv.zip')
test_labels = pd.read_csv('/kaggle/input/jigsaw-toxic-comment-classification-challenge/test_labels.csv.zip')
# subm = pd.read_csv('../input/sample_submission.csv')

In [6]:
import re, string
re_tok = re.compile(f'([{string.punctuation}“”¨«»®´·º½¾¿¡§£₤‘’\n])')
def clean_string(s): return re_tok.sub(' ', s)

In [7]:
class MyData(torch.utils.data.Dataset):
    def __init__(self, data, label_cols):
        self.data = data
        self.label_cols = label_cols #test comment

    def __getitem__(self, item):

        comment = clean_string(self.data.comment_text[item])
        toxic = self.data.toxic[item]
        severe_toxic = self.data.severe_toxic[item]
        obscene = self.data.obscene[item]
        threat = self.data.threat[item]
        insult = self.data.insult[item]
        identity_hate = self.data.identity_hate[item]
#         none= self.data.none[item]
        return comment, torch.FloatTensor([toxic, severe_toxic, obscene, insult, identity_hate])
    def __len__(self):
        return len(self.data)

In [8]:
label_cols = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
# train['none'] = 1-train[label_cols].max(axis=1)
# label_cols.append('none')
COMMENT = 'comment_text'
label_cols.append(COMMENT)
train[COMMENT].fillna("unknown", inplace=True)
test[COMMENT].fillna("unknown", inplace=True)

In [9]:
gpu = 0
device = torch.device(gpu if torch.cuda.is_available() else "cpu")
if torch.cuda.is_available():
    torch.cuda.set_device(gpu)
print(device)

cuda:0


In [10]:
MAX_SEQ_LEN=512
BATCH_SIZE =4
LEARNING_RATE = 1e-3
WARMUP_STEPS = 4
EPOCHS = 2
model_name= 'roberta-base'
# wandb.init(project=model_name)
# config = wandb.config

In [11]:
# tokenizer = AutoTokenizer.from_pretrained(model_name, model_max_length=MAX_SEQ_LEN)
import transformers
tokenizer = AutoTokenizer.from_pretrained(model_name, model_max_length=MAX_SEQ_LEN)
tokenizer.add_special_tokens({'pad_token': '[PAD]'})

1

In [12]:

class AutoNet(nn.Module):
  def __init__(self, seqLength=MAX_SEQ_LEN, numClasses=5, model_name=model_name, len_t = len(tokenizer)):
    super(AutoNet, self).__init__()
    self.model = AutoModel.from_pretrained(model_name)
    self.model.resize_token_embeddings(len_t)
    # for p in self.bert.parameters():
    #   p.requires_grad=False
    self.fc = nn.Linear(seqLength*768,numClasses)
    # self.drop = nn.Dropout(p=0.2)
    self.sigmoid = nn.Sigmoid()
  def forward(self,input_ids, attention_mask):
    x = self.model(input_ids, attention_mask=attention_mask)[0] #last_hidden_state #BSD

    x = self.fc(x.view(x.shape[0],-1))
    # x = self.drop(x)
    # x = F.softmax(x, dim=1)
    x = self.sigmoid(x)
    return x

In [13]:
train_set = MyData(train, label_cols)
train_set, val_set = torch.utils.data.random_split(train_set, [int(0.8*len(train_set)), len(train_set)-int(0.8*len(train_set))] )
train_set = torch.utils.data.DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=True, num_workers=4)
val_set = torch.utils.data.DataLoader(val_set, batch_size=BATCH_SIZE, shuffle=True, num_workers=4)

In [15]:
model = AutoNet()
model = model.to(device)
model.train()
optimizer = AdamW(model.parameters(), lr=LEARNING_RATE)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=WARMUP_STEPS, num_training_steps = -1)
loss_criteria = nn.MSELoss()#nn.BCELoss()
loss_criteria = loss_criteria.to(device)
model.load_state_dict(torch.load(f'/kaggle/input/notebook294d5a41bc-f62f18-gpt/roberta-baseNetMLE_2.pt'))

<All keys matched successfully>

In [16]:
from sklearn.metrics import accuracy_score


In [ ]:

model.train()
from tqdm import tqdm
for epoch in tqdm(range(1)):
  count = 0
  total_loss =0 
  model.train()
  for i,data in enumerate(train_set):
    optimizer.zero_grad()
    enc = tokenizer.batch_encode_plus(list(data[0]), pad_to_max_length=True, max_length=MAX_SEQ_LEN, return_tensors='pt')
    input_ids = enc['input_ids'].to(device)
    attention_mask = enc['attention_mask'].to(device)
    labels = torch.tensor(data[1]).to(device)
    out = model(input_ids=input_ids, attention_mask=attention_mask)
    # loss, logits = out[:2]
    # print(out.shape, logits.shape, labels.shape)
    loss = loss_criteria(out, labels)
    loss.backward()
    optimizer.step()
    scheduler.step()
    total_loss += loss.detach().data
    if (i+1)%1024==0:
      print(f"Epoch: {epoch}, batch: {i+1}, loss: {total_loss/(1024*BATCH_SIZE)}")
#       wandb.log({"train_loss_per_batch": loss})
      total_loss = 0
  torch.save(model.state_dict(), f'{model_name}NetMLE_2.pt')

  model.eval()
  all_pred = []
  all_gold = []
  for i,data in enumerate(val_set):
    enc = tokenizer.batch_encode_plus(list(data[0]), pad_to_max_length=True, max_length=MAX_SEQ_LEN, return_tensors='pt')
    input_ids = enc['input_ids'].to(device)
    attention_mask = enc['attention_mask'].to(device)
    labels = torch.tensor(data[1]).to(device)
    out = model(input_ids=input_ids, attention_mask=attention_mask)
    all_pred.extend(1*(out>0.8).clone().detach().cpu().numpy())
    all_gold.extend((labels.type(torch.LongTensor).detach().cpu().numpy()))
    
  count=0
  for i in range(len(all_gold)):
    if (all_gold[i]==all_pred[i]).all():
      count+=1
#   wandb.log({"val_acc": count/len(all_gold)})
  print("Validation accuracy:", count/len(all_gold))

In [ ]:
# model.load_state_dict(torch.load(f'/kaggle/input/bertnet/BertNetCE_2.pt'))
model.eval()
all_pred = []
all_gold = []
for i,data in enumerate(val_set):
#     optimizer.zero_grad()
    enc = tokenizer.batch_encode_plus(list(data[0]), pad_to_max_length=True, max_length=MAX_SEQ_LEN, return_tensors='pt')
    input_ids = enc['input_ids'].to(device)
    attention_mask = enc['attention_mask'].to(device)
    labels = torch.tensor(data[1]).to(device)
    out = model(input_ids=input_ids, attention_mask=attention_mask)

    all_pred.extend(1*(out>0.8).clone().detach().cpu().numpy())
    all_gold.extend((labels.type(torch.LongTensor).detach().cpu().numpy()))


In [17]:
from sklearn.metrics import accuracy_score
accuracy_score(all_gold, all_pred)

NameError: name 'all_gold' is not defined

In [18]:
label_cols

['toxic',
 'severe_toxic',
 'obscene',
 'threat',
 'insult',
 'identity_hate',
 'comment_text']

In [ ]:
# label_cols.remove('comment_text')  

In [ ]:
# test_labels = test_labels[label_cols]+1

In [24]:
test_labels = test_labels[test_labels.toxic!=-1]
test_labels.sample(10)
# a = test[:30]
# print(a)
# a.to_csv(r'sample_debug.csv', index = False)

                  id                                       comment_text
0   00001cee341fdb12  Yo bitch Ja Rule is more succesful then you'll...
1   0000247867823ef7  == From RfC == \n\n The title is fine as it is...
2   00013b17ad220c46  " \n\n == Sources == \n\n * Zawe Ashton on Lap...
3   00017563c3f7919a  :If you have a look back at the source, the in...
4   00017695ad8997eb          I don't anonymously edit articles at all.
5   0001ea8717f6de06  Thank you for understanding. I think very high...
6   00024115d4cbde0f  Please do not add nonsense to Wikipedia. Such ...
7   000247e83dcc1211                   :Dear god this site is horrible.
8   00025358d4737918  " \n Only a fool can believe in such numbers. ...
9   00026d1092fe71cc  == Double Redirects == \n\n When fixing double...
10  0002eadc3b301559  I think its crap that the link to roggenbier i...
11  0002f87b16116a7f  "::: Somebody will invariably try to add Relig...
12  0003806b11932181  , 25 February 2010 (UTC) \n\n :::Looking i

In [ ]:
test = pd.read_csv('/kaggle/input/jigsaw-toxic-comment-classification-challenge/train.csv.zip')
test_set = test.merge(test_labels, left_index=True, right_index=True)
test_set = test_set[["id_x", "comment_text", "toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]]
test_set = test_set.reset_index(drop=True)
test_set = test_set.rename(columns={"id_x": "id"})

# test_set['none'] = 1-test_set[label_cols].max(axis=1)

In [ ]:
test_set.head()

In [ ]:
test_set = MyData(test_set, label_cols)
test_set = torch.utils.data.DataLoader(test_set, batch_size=BATCH_SIZE, shuffle=True, num_workers=4)

In [ ]:
# model.load_state_dict(torch.load(f'/kaggle/input/bertnet/BertNetCE_2.pt'))
model.eval()
all_pred = []
all_gold = []
for i,data in enumerate(test_set):
#     optimizer.zero_grad()
    enc = tokenizer.batch_encode_plus(list(data[0]), pad_to_max_length=True, max_length=MAX_SEQ_LEN, return_tensors='pt')
    input_ids = enc['input_ids'].to(device)
    attention_mask = enc['attention_mask'].to(device)
    labels = torch.tensor(data[1]).to(device)
    out = model(input_ids=input_ids, attention_mask=attention_mask)

    all_pred.extend(1*(out>0.8).clone().detach().cpu().numpy())
    all_gold.extend((labels.type(torch.LongTensor).detach().cpu().numpy()))


In [ ]:
len(all_gold)

In [ ]:
accuracy_score(all_gold, all_pred)  

In [ ]:
count=0
for i in range(len(all_gold)):
    if (all_gold[i]==all_pred[i]).all():
        count+=1
count/len(all_gold)

In [ ]:
# wandb.log({"test_acc": count/len(all_gold) })